# Attack on scAnnotatR

- How to train a scAnnotatR classifier
- How to format the classifier to use it with adverSCarial
- How to run an CGD attack

Nguyen, V., Griss, J. scAnnotatR: framework to accurately classify cell types in single-cell RNA-sequencing data. BMC Bioinformatics, 2022. 23(44) https://doi.org/10.1186/s12859-022-04574-5 

In [1]:
library(scAnnotatR)
library(IRdisplay)
library(adverSCarial)

Loading required package: Seurat

Loading required package: SeuratObject

Loading required package: sp

The legacy packages maptools, rgdal, and rgeos, underpinning this package
will retire shortly. Please refer to R-spatial evolution reports on
https://r-spatial.org/r/2023/05/15/evolution4.html for details.
This package is now running under evolution status 0 

‘SeuratObject’ was built under R 4.3.0 but the current version is
4.3.3; it is recomended that you reinstall ‘SeuratObject’ as the ABI
for R may have changed


Attaching package: ‘SeuratObject’


The following object is masked from ‘package:base’:

    intersect


Loading required package: SingleCellExperiment

Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, c

# scAnnotatR classifier training

Load previously splitted train/test pbmc3k dataset
https://cf.10xgenomics.com/samples/cell/pbmc3k/pbmc3k_filtered_gene_bc_matrices.tar.gz

annotated from the Seurat tutorial method
https://satijalab.org/seurat/articles/pbmc3k_tutorial

In [2]:
se_train <- readRDS("data//v2/pbmc_train.rds")
se_test <- readRDS("data//v2/pbmc_test.rds")

In [3]:
Idents(se_train) <- "chr_seurat_cluster"
Idents(se_test) <- "chr_seurat_cluster"

In [4]:
head(se_train@meta.data)

,orig.ident,nCount_RNA,nFeature_RNA,percent.mt,RNA_snn_res.0.5,seurat_clusters,chr_seurat_cluster
,<fct>,<dbl>,<int>,<dbl>,<fct>,<fct>,<chr>
AAACGCTGGTTCTT-1,pbmc3k,2260,790,3.097345,4,4,CD8 T
AAACGCTGTAGCCA-1,pbmc3k,1275,532,1.176471,0,0,Naive CD4 T
AAACGCTGTTTCTG-1,pbmc3k,1103,550,2.901179,5,5,FCGR3A+ Mono
AAACTTGATCCAGA-1,pbmc3k,2388,747,1.088777,0,0,Naive CD4 T
AAAGAGACGCGAGA-1,pbmc3k,3033,1058,1.417738,1,1,CD14+ Mono
AAAGAGACGGACTT-1,pbmc3k,1151,457,2.345786,0,0,Naive CD4 T


## Function to get the most significant genes between a cluster and the other cells

In [14]:
getSignGenesNot <- function(expr, clusters, target, method="wilcox", adjMethod="BH", verbose=FALSE){
        if (verbose) {message("Cluster ",target," vs all the other clusters")}
        pvals <- apply(t(expr), 1, function(x){
            c1 <- x[clusters == target]
            c1 <- c1[!is.na(c1)]
            c2 <- x[clusters != target]
            c2 <- c2[!is.na(c2)]
            if ( length(c1) == 0 || length(c2) == 0){
                return(1)
            }
            if (length(unique(c1))==1){
                c1[1] = c1[1] + 0.00001
            }
            if (length(unique(c2))==1){
                c2[1] = c2[1] + 0.00001
            }
            if (method=="wilcox"){
                return(wilcox.test(c1, c2)$p.value)
            }
            if (method=="ttest"){
                return(t.test(c1, c2)$p.value)
            }
        })
        means <- apply(t(expr), 1, function(x){
                c1 <- x[clusters == target]
                c1 <- c1[!is.na(c1)]
                c2 <- x[clusters != target]
                c2 <- c2[!is.na(c2)]
                if (length(c1) == 0){
                    c1 = c(0)
                }
                if (length(c2) == 0){
                    c2 = c(0)
                }
                return(mean(c1)-mean(c2))
        })  
        dfPvals <- data.frame(gene=colnames(expr), pval=unname(pvals), mean=means)
        rownames(dfPvals) <- dfPvals$gene

        for (clustInt in setdiff(unique(clusters), target)){
            if(verbose){message("Cluster ", target, " vs cluster ", clustInt)}
            newPvals <- pvals <- apply(t(expr), 1, function(x){
                c1 <- x[clusters == target]
                c1 <- c1[!is.na(c1)]
                c2 <- x[clusters == clustInt]
                c2 <- c2[!is.na(c2)]
                if ( length(c1) == 0 || length(c2) == 0){
                    return(1)
                }
                if (length(unique(c1))==1){
                    c1[1] = c1[1] + 0.001
                }
                if (length(unique(c2))==1){
                    c2[1] = c2[1] + 0.001
                }
                if (method=="wilcox"){
                    return(wilcox.test(c1, c2)$p.value)
                }
                if (method=="ttest"){
                    return(t.test(c1, c2)$p.value)
                }    
            })
            newMeans <- pvals <- apply(t(expr), 1, function(x){
                c1 <- x[clusters == target]
                c1 <- c1[!is.na(c1)]
                c2 <- x[clusters == clustInt]
                c2 <- c2[!is.na(c2)]
                if (length(c1) == 0){
                    c1 = c(0)
                }
                if (length(c2) == 0){
                    c2 = c(0)
                }
                return(mean(c1)-mean(c2))
            })  
            if(verbose){message(sum(unname(newPvals) < dfPvals$pval)," pvalues replaced by lower values")}
            dfPvals[unname(newPvals) < dfPvals$pval, "pval"] <- newPvals[unname(newPvals) < dfPvals$pval]
            dfPvals[unname(newPvals) < dfPvals$pval, "mean"] <- newMeans[unname(newPvals) < dfPvals$pval]
    }
    dfPvals$adjPval <- p.adjust(dfPvals$pval, method=adjMethod)
    dfPvals <- dfPvals[order(dfPvals$pval),]
    return(dfPvals)
}

In [5]:
dfScaled <- as.data.frame(se_train@assays$RNA@layers$scale.data)
colnames(dfScaled) <- rownames(se_train@assays$RNA@cells)
rownames(dfScaled) <- rownames(se_train@assays$RNA@features)
dfScaled <- as.data.frame(t(dfScaled))

The scAnnotatR will build one classifier to predict each cell type, so we one-hot encode the cell type by adding a meta.data column for each cell type with binary information: cell type or "unknown".

In [22]:
for (cellType in unique(se_train@meta.data$chr_seurat_cluster)){
    se_train@meta.data[[cellType]] <- unlist(lapply(se_train@meta.data$chr_seurat_cluster, function(x){
        if (x == cellType){
            return(cellType)
        } else {
            return("unknown")
        }
    }))
}

## Train and export one classifier for each cell type

In [23]:
listClassifiers <- list()
for (cellType in unique(se_train@meta.data$chr_seurat_cluster)){
    display(cellType)
    sg <- getSignGenesNot(dfScaled,
                   se_train@meta.data$chr_seurat_cluster,
                   cellType)
    # Selection of the 20 most significants with a minimum mean difference of 0.5 for the SVM
    selectedMarkers <- rownames(sg[abs(sg$mean)>0.5,])[1:20]
    classifier <- train_classifier(train_obj = se_train, cell_type = cellType, 
                                 marker_genes = selectedMarkers,
                                 assay = 'RNA', tag_slot = cellType)
    listClassifiers[[cellType]] <- classifier
    
    save_new_model(new_model = classifier, path_to_models = "repr_data/classifiers/scAnnotatR/trainedModels",
               include.default = FALSE) 
}
listClassifiers[[cellType]]

[1] "CD8 T"

Loading required package: ggplot2

Loading required package: lattice

Saving new models to repr_data//classifiers//scAnnotatR//trainedModels/new_models.rda...

Finished saving new model



[1] "Naive CD4 T"

Saving new models to repr_data//classifiers//scAnnotatR//trainedModels/new_models.rda...

Finished saving new model



[1] "FCGR3A+ Mono"

...

[1] "DC"

Saving new models to repr_data//classifiers//scAnnotatR//trainedModels/new_models.rda...

Finished saving new model



[1] "Platelet"

“There were missing values in resampled performance measures.”
Saving new models to repr_data//classifiers//scAnnotatR//trainedModels/new_models.rda...

Finished saving new model



An object of class scAnnotatR for Platelet 
* 20 marker genes applied: C2orf88, TMEM40, HGD, GP9, PF4, SPARC, LY6G6F, TREML1, PTCRA, AC147651.3, GNG11, AP001189.4, CMTM5, GP1BA, ITGA2B, ITGB3, RP11-879F14.2, CLDN5, SEPT5, PVALB 
* Predicting probability threshold: 0.5 
* No parent model

# Format the Classifier
To work with adverSCarial the classifier needs to be formated in a certain way.

In [24]:
scAnnotatR_classifier = function(expr, clusters, target){
    if (!"scAnnotatR" %in% loadedNamespaces()){
        library(scAnnotatR)
    }
    if ( !exists("sca_default_models")){
        sca_default_models <<- load_models("repr_data/classifiers/scAnnotatR/trainedModels")
        sca_cell_types <<- names(sca_default_models)
    }
    
    seurat.obj <- classify_cells(classify_obj = expr, 
                             assay = 'RNA', slot = 'scale.data',
                             cell_types = sca_cell_types, 
                             path_to_models = "repr_data/classifiers/scAnnotatR/trainedModels")
    typePredictions <- seurat.obj@meta.data[, stringr::str_replace_all( paste0(sca_cell_types, "_p"), " ", "_")]
    
    pred_table <- table(seurat.obj@meta.data[clusters == target, "most_probable_cell_type"])
    cluster_pred <- names(pred_table[order(pred_table, decreasing = TRUE)])[1]
    odd <- unname(pred_table[cluster_pred]/sum(pred_table))
    colnames(typePredictions) <- unlist(lapply(colnames(typePredictions), function(x){
        unlist(strsplit(x,"_p$"))[1]
    }))
    typePredictions <- as.data.frame(t(typePredictions))
    
    result <- list(
        # Cell type prediction for the cluster
        prediction=stringr::str_replace_all(cluster_pred," ","_"),
        # Score of the predicted cell type
        odd=odd, 
        # Score for each cell type for each cell
        typePredictions=typePredictions,
        # Cell type for each cell
        cellTypes=seurat.obj@meta.data$most_probable_cell_type)
    return(result)
}

# Adversarial attack of the NK (Natural Killer) cells cluster

In [25]:
so_pbmc_test <- se_test
clusters_so = so_pbmc_test@meta.data$chr_seurat_cluster
names(clusters_so) <- rownames(so_pbmc_test@meta.data)

cell types clusters

In [26]:
unique(clusters_so)

[1] "Memory CD4 T" "B"            "CD14+ Mono"   "NK"           "CD8 T"       
[6] "Naive CD4 T"  "FCGR3A+ Mono" "DC"           "Platelet"

We check if the classifier is working properly, it predicts the NK cells cluster as NK cells

In [48]:
class_results <- scAnnotatR_classifier(so_pbmc_test, clusters_so, 'NK')

In [49]:
class_results$prediction

[1] "NK"

In [36]:
dfScaled <- as.data.frame(so_pbmc_test@assays$RNA@layers$scale.data)
colnames(dfScaled) <- rownames(so_pbmc_test@assays$RNA@cells)
rownames(dfScaled) <- rownames(so_pbmc_test@assays$RNA@features)
dfScaled <- t(dfScaled)

In [37]:
dfScaled[1:5,1:5]

,AL627309.1,AP006222.2,RP11-206L10.2,RP11-206L10.9,LINC00115
AAACATACAACCAC-1,-0.05812316,-0.03357571,-0.04166819,-0.03364562,-0.08223981
AAACATTGAGCTAC-1,-0.05812316,-0.03357571,-0.04166819,-0.03364562,-0.08223981
AAACATTGATCAGC-1,-0.05812316,-0.03357571,-0.04166819,-0.03364562,-0.08223981
AAACCGTGCTTCCG-1,-0.05812316,-0.03357571,-0.04166819,-0.03364562,-0.08223981
AAACCGTGTATGCG-1,-0.05812316,-0.03357571,-0.04166819,-0.03364562,-0.08223981


We use the adverSCarial getSignGenes function to get the most significant genes between clusters, ensuring that all pairs of clusters are equivalently representated.

In [38]:
sign_genes <- getSignGenes(dfScaled, clusters_so)

In [40]:
head(sign_genes$results)

,gene,pval
,<chr>,<dbl>
HLA-DRA,HLA-DRA,1.004947e-70
PRF1,PRF1,6.443834e-62
NKG7,NKG7,1.070165e-79
FCER1A,FCER1A,2.765930e-18
TYROBP,TYROBP,8.127634e-85
IL32,IL32,1.260937e-66


We launch the attack with the advCGD function with alpha=2 and epsilon=2 parameters, which lead to a big modification on a few genes. You can try with alpha=1 and epsilon=1 to have small modifications of more genes.

In [43]:
cgd_results_nk <- advCGD(so_pbmc_test, clusters_so, 'NK',
                          scAnnotatR_classifier, alpha=2,
                          epsilon=2, slot="scale.data",
                          genes=sign_genes$results$gene[1:500],
                         verbose=T)

NK

13189

1318

72

New cluster target: CD8_T

HLA-DRA 1

Number of original annot NK : 70

 mean 0.987004631106661 delt 0

Number of CD8_T : 2

 mean 0.958532596757498 delt 0

Number of modified cells 0

PRF1 2

Number of original annot NK : 70

 mean 0.987004631106661 delt 0

Number of CD8_T : 2

 mean 0.958532596757498 delt 0

Number of modified cells 70

NKG7 3

Number of original annot NK : 70

 mean 0.957668058445093 delt -0.0293365726615682

Number of CD8_T : 2

 mean 0.958532596757498 delt 0

Number of modified cells 70

FCER1A 4

Number of original annot NK : 71

 mean 0.965101381254414 delt 0.0074333228093213

Number of CD8_T : 1

 mean 0.975073064539234 delt 0.0165404677817356

Number of modified cells 0

TYROBP 5

Number of original annot NK : 71

 mean 0.965101381254414 delt 0

Number of CD8_T : 1

 mean 0.975073064539234 delt 0

Number of modified cells 71



We can see the genes that have been modified

In [50]:
cgd_results_nk$modGenes

[1] "PRF1"   "NKG7"   "TYROBP"

And the modified Seurat object

In [52]:
cgd_results_nk$expr

An object of class Seurat 
13714 features across 1318 samples within 1 assay 
Active assay: RNA (13714 features, 2000 variable features)
 3 layers present: counts, data, scale.data
 2 dimensional reductions calculated: pca, umap

The NK cluster is now classified as CD8 T cells

In [46]:
new_classification <- scAnnotatR_classifier(cgd_results_nk$expr, clusters_so, 'NK')
new_classification$prediction

[1] "CD8_T"

In [55]:
sessionInfo()

R version 4.3.3 (2024-02-29)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Ubuntu 20.04.4 LTS

Matrix products: default
BLAS:   /usr/lib/x86_64-linux-gnu/blas/libblas.so.3.9.0 
LAPACK: /usr/lib/x86_64-linux-gnu/lapack/liblapack.so.3.9.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=fr_FR.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=fr_FR.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=fr_FR.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=fr_FR.UTF-8 LC_IDENTIFICATION=C       

time zone: Europe/Paris
tzcode source: system (glibc)

attached base packages:
[1] stats4    stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] caret_6.0-94                lattice_0.21-8             
 [3] ggplot2_3.4.4               adverSCarial_1.3.3         
 [5] IRdisplay_1.1               scAnnotatR_1.8.0           
 [7] SingleCell